In [72]:
import numpy as np
import pandas as pd
import ast
import re

In [74]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')
moviesp = pd.read_csv('movies_production.csv')

In [75]:
moviesp.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,production
0,19995,Avatar,"['In', 'the', '22nd', 'century,', 'a', 'parapl...","['Action', 'Adventure', 'Fantasy', 'ScienceFic...","['cultureclash', 'future', 'spacewar', 'spacec...","['SamWorthington', 'ZoeSaldana', 'SigourneyWea...",['JamesCameron'],"['DuneEntertainment', 'LightstormEntertainment..."


In [77]:
moviesp['tags'] = moviesp['overview'] + moviesp['genres'] + moviesp['keywords'] + moviesp['cast'] + moviesp['crew'] + moviesp['production']

In [80]:
moviesp.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,production,tags
0,19995,Avatar,"['In', 'the', '22nd', 'century,', 'a', 'parapl...","['Action', 'Adventure', 'Fantasy', 'ScienceFic...","['cultureclash', 'future', 'spacewar', 'spacec...","['SamWorthington', 'ZoeSaldana', 'SigourneyWea...",['JamesCameron'],"['DuneEntertainment', 'LightstormEntertainment...","['In', 'the', '22nd', 'century,', 'a', 'parapl..."


In [82]:
new_df = moviesp[['movie_id','title','tags']]

In [84]:
new_df['tags'][0]

"['In', 'the', '22nd', 'century,', 'a', 'paraplegic', 'Marine', 'is', 'dispatched', 'to', 'the', 'moon', 'Pandora', 'on', 'a', 'unique', 'mission,', 'but', 'becomes', 'torn', 'between', 'following', 'orders', 'and', 'protecting', 'an', 'alien', 'civilization.']['Action', 'Adventure', 'Fantasy', 'ScienceFiction']['cultureclash', 'future', 'spacewar', 'spacecolony', 'society', 'spacetravel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alienplanet', 'cgi', 'marine', 'soldier', 'battle', 'loveaffair', 'antiwar', 'powerrelations', 'mindandsoul', '3d']['SamWorthington', 'ZoeSaldana', 'SigourneyWeaver']['JamesCameron']['DuneEntertainment', 'LightstormEntertainment', '20thCenturyFox', 'IngeniousMedia']"

In [88]:
import re

new_df['tags'] = new_df['tags'].apply(
    lambda x: " ".join(re.findall(r"[A-Za-z0-9]+", x))
)

C:\Users\SUCCESS\AppData\Local\Temp\ipykernel_28360\3773906096.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(


In [90]:
new_df['tags']

0       In the 22nd century a paraplegic Marine is dis...
1       Captain Barbossa long believed to be dead has ...
2       A cryptic message from Bond s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war weary former military cap...
                              ...                        
4801    El Mariachi just wants to play his guitar and ...
4802    A newlywed couple s honeymoon is upended by th...
4803    Signed Sealed Delivered introduces a dedicated...
4804    When ambitious New York attorney Sam is sent t...
4805    Ever since the second grade when he first saw ...
Name: tags, Length: 4806, dtype: object

In [92]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\SUCCESS\AppData\Local\Temp\ipykernel_28360\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [94]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,in the 22nd century a paraplegic marine is dis...
1,285,Pirates of the Caribbean: At World's End,captain barbossa long believed to be dead has ...
2,206647,Spectre,a cryptic message from bond s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,john carter is a war weary former military cap...


In [96]:
import nltk

In [98]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [100]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert all movie tags to embeddings
embeddings = model.encode(new_df['tags'].tolist())

C:\Users\SUCCESS\anaconda3\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [102]:
# Compute similarity matrix
similarity = cosine_similarity(embeddings)

In [104]:
def recommend(movie):
    # 1️⃣ Find index of the movie
    movie_index = new_df[new_df['title']==movie].index[0]
    print(movie_index)
    
    # 2️⃣ Get similarity scores for this movie
    distances = similarity[movie_index]
    
    # 3️⃣ Sort and get top 5 similar movies (skip the movie itself)
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    # 4️⃣ Print recommended movie titles
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [116]:
recommend('The Godfather')

3341
The Godfather: Part II
The Godfather: Part III
GoodFellas
The Conformist
The Funeral


In [112]:
pickle.dump(new_df.to_dict(), open('movies_perfect.pkl','wb'))

In [114]:
pickle.dump(similarity, open('similarity_perfect.pkl','wb'))